# Oversampling simple duplicando filas de la clase "experto"

En esta celda realizamos oversampling manual para balancear el dataset, específicamente para la clase minoritaria `"experto"`. 

**Pasos:**
- Se carga el dataset original `musical_skills_dataset.csv`.
- Se separan las filas con `skill_level` igual a `"experto"`.
- Se calcula cuál es la clase mayoritaria entre `"intermedio"` y `"principiante"`.
- Se duplican aleatoriamente filas de `"experto"` hasta igualar la cantidad de la clase mayoritaria.
- Se concatenan los datos y se guarda el nuevo dataset balanceado en `musical_skills_oversampled_simple.csv`.

Este método es sencillo y rápido, pero puede causar sobreajuste porque duplica datos existentes sin crear nuevos ejemplos.

In [1]:
# Celda 1: Oversampling simple duplicando filas "experto"
import pandas as pd
from sklearn.utils import resample

# Cargar dataset
df = pd.read_csv('musical_skills_dataset.csv')

# Separar clases
df_experto = df[df['skill_level'] == 'experto']
df_others = df[df['skill_level'] != 'experto']

# Cantidades actuales
n_intermedio = len(df[df['skill_level'] == 'intermedio'])
n_principiante = len(df[df['skill_level'] == 'principiante'])
n_experto = len(df_experto)

# Queremos que experto tenga la misma cantidad que la clase más grande
max_count = max(n_intermedio, n_principiante)

# Upsample experto duplicando aleatoriamente
df_experto_upsampled = resample(df_experto, 
                                replace=True,     # muestra con reemplazo
                                n_samples=max_count,    # igualar tamaño a la clase mayor
                                random_state=42)

# Concatenar el dataset balanceado
df_balanced = pd.concat([df_others, df_experto_upsampled])

# Guardar en un nuevo archivo CSV
df_balanced.to_csv('musical_skills_oversampled_simple.csv', index=False)

print(f"Original experto: {n_experto}, Upsampled experto: {len(df_experto_upsampled)}")
print("Archivo guardado como musical_skills_oversampled_simple.csv")

Original experto: 151, Upsampled experto: 452
Archivo guardado como musical_skills_oversampled_simple.csv


# Oversampling con SMOTE (Synthetic Minority Over-sampling Technique)

Esta celda aplica SMOTE, una técnica de oversampling que genera nuevos ejemplos sintéticos para la clase minoritaria basándose en interpolación de sus vecinos más cercanos.

**Pasos:**
- Cargar el dataset original.
- Codificar la variable categórica `skill_level` en números para que SMOTE pueda procesarla.
- Separar las características (`X`) y la variable objetivo codificada (`y`).
- Aplicar SMOTE para balancear las clases automáticamente.
- Reconstruir un DataFrame con los datos balanceados.
- Decodificar la variable objetivo para regresar a sus etiquetas originales.
- Guardar el dataset balanceado en `musical_skills_oversampled_smote.csv`.

SMOTE suele ayudar a evitar el sobreajuste que ocurre con la simple duplicación, ya que genera datos sintéticos más variados.

In [1]:
# Celda 2: Oversampling con SMOTE
import pandas as pd
from imblearn.over_sampling import SMOTE

# Cargar dataset
df = pd.read_csv('musical_skills_dataset.csv')

# Convertir variable categórica 'skill_level' a números
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['skill_level_encoded'] = le.fit_transform(df['skill_level'])

# Separar características y target
X = df.drop(columns=['skill_level', 'skill_level_encoded'])
y = df['skill_level_encoded']

# Aplicar SMOTE para balancear la clase minoritaria
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X, y)

# Reconstruir DataFrame con los datos resampleados
df_resampled = X_res.copy()
df_resampled['skill_level_encoded'] = y_res

# Decodificar skill_level
df_resampled['skill_level'] = le.inverse_transform(df_resampled['skill_level_encoded'].astype(int))

# Reordenar columnas para que coincidan con original
cols = ['skill_level'] + [c for c in df_resampled.columns if c not in ['skill_level', 'skill_level_encoded']] 
df_resampled = df_resampled[cols]

# Guardar a CSV
df_resampled.to_csv('musical_skills_oversampled_smote.csv', index=False)

print("Archivo guardado como musical_skills_oversampled_smote.csv")

Archivo guardado como musical_skills_oversampled_smote.csv
